<a href="https://colab.research.google.com/github/adimis-ai/Intelligent-Document-Search-and-Question-Answering-with-LangChain/blob/main/langchain_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install requests transformers faiss-cpu
!pip install PyPDF2

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.docstore.document import Document
import requests
from google.colab import drive
import os
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter
import pickle
import os


In [ ]:
%env OPENAI_API_KEY = <your key here. NO QUOTES>

In [ ]:
drive.mount('/content/drive')
gdrive_path = '/content/drive/MyDrive/sleep_pdf/'

In [ ]:
def get_pdf_data(file_path, num_pages = 1):
  reader = PdfReader(gdrive_path+file_path)
  full_doc_text = ""
  for page in range(len(reader.pages)):
    current_page = reader.pages[page]
    text = current_page.extract_text()
    full_doc_text += text


  return Document(
        page_content=full_doc_text,
        metadata = {"source": file_path}
    )

In [ ]:
def source_docs():
    return [get_pdf_data(file) for file in os.listdir(gdrive_path)]

In [ ]:
def search_index(source_docs):
    source_chunks = []
    splitter = CharacterTextSplitter(separator=" ", chunk_size=1024, chunk_overlap=0)

    for source in source_docs:
        for chunk in splitter.split_text(source.page_content):
            source_chunks.append(Document(page_content=chunk, metadata=source.metadata))
    with open("search_index.pickle", "wb") as f:
        pickle.dump(FAISS.from_documents(source_chunks, OpenAIEmbeddings()), f)

In [ ]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0),verbose=False, chain_type="stuff")
def print_answer(question):
    with open("search_index.pickle", "rb") as f:
        search_index = pickle.load(f)
    print(
        chain(
            {
                "input_documents": search_index.similarity_search(question, k=3),
                "question": question,
            },
            return_only_outputs=True,
        )["output_text"]

    )


In [ ]:
import pdb
sources = source_docs()
search_index(sources)
try:
  print_answer(<your question on the document corpus here>) # your question here
except Exception as e:
  print(f"Unexpected error: {e}")